# Filtrado Colaborativo: *Probabilistic Matrix Factorization*

Durante los inicios del filtrado colaborativo el algoritmo de KNN era el más empleado debido a los buenos resultados que reportaba y a la facilidad con la que podían explicarse sus recomendaciones. Sin embargo, este algoritmo tiene una gran desventaja: su escalabilidad. El algoritmo de KNN funciona bien para datasets de tamaño medio, pero, a medida que el dataset crece, los tiempos de cómputo para obtener las recomendaciones se vuelven inasumibles. Aumentar el número de usuarios y/o el número de items implicar ralentizar el cálculo de las similaridades, la búsqueda de los k vecinos y el número de predicciones a realizar.

Como consecuencia de estos problema y del gran empuje que supuso el [Netflix Prize](https://www.netflixprize.com/) (concurso que ofrecía una recompensa de 1M de dólares al equipo que consiguiera mejorar el RMSE en el dataset de Netflix) comenzaron a ganar fuerza los sistemas de filtrado colaborativo basados en modelos, más concretamente los basados en modelos de factorización matricial.

El **filtrado colaborativo basado en factorización matricial** se basa en la siguiente idea: las votaciones que los usuarios realizan a los items están condicionadas por una serie de factores latentes intrínsecos a los usuarios y los items. Ilustremos esto con un ejemplo. Supongamos un sistema de recomendación de películas. Lo que postula la factorización matricial es que los usuarios votan las películas basándose no sólo en la propia película, sino que lo hacen basándose en las características que describen esa película. Si a un usuario le gustan las películas de acción con un toque de comedia, es muy probable que le gusten todas las películas de acción con un toque de comedia. Los algoritmos de filtrado colaborativo buscan estas propiedades intrínsecas al dominio en el que se realizan las recomendaciones y las denominan **factores latentes** u ocultos. Es importante resaltar que estos factores son ocultos, y aunque en el ejemplo de la recomendación de películas podamos suponer que se trata de géneros de cine, el modelo nunca nos va a indicar con qué género se corresponde cada factor.

Matemáticamente, la factorización matricial consiste en encontrar las matrices $P$ y $Q$ que satisfagan la siguiente expresión:

$$R \approx P \cdot Q$$

En esta expresión:

- $R$ representa la matriz (dispersa) con las votaciones de los usuarios (filas) a los items (columnas).
- $P$ representa las matriz (densa) de factores de los usuarios (filas) con los *k* factores latentes (columnas).
- $Q$ representa las matriz (densa) de factores de los items (columnas) con los *k* factores latentes (filas).

Como vemos, los modelos tienen un parámetro que será necesario tunear con el fin de ajustar el modelo a cada dataset. Este parámetro ***k*** representa el número de factores latentes de nuestro modelo.

Desarrollando la expresión anterior, podemos inferir que predicción de voto de un usuario $u$ a un item $i$ queda como:

$$\hat{r}_{u,i} = \vec{p}_u \cdot \vec{q}_i$$

Dónde $\vec{p}_u$ representa un vector fila de la matriz $P$ con los factores latentes del usuario $u$ y $\vec{q}_i$ representa un vector columna de la matriz $Q$ con los factores latentes del item $i$.

Por lo tanto, podemos plantear la búsqueda de los factores latentes como un problema de optimización, en el cual buscamos minimizar el error cometido en los votos conocidos:

$$\min_{p,q} \sum_{(u,i) \in R} ( r_{u,i} - \vec{p}_u \cdot \vec{q}_i)^2$$

Expresión a la que podemos añadir una regularización para evitar el *overfitting*:

$$\min_{p,q} \sum_{(u,i) \in R} ( r_{u,i} - \vec{p}_u \cdot \vec{q}_i)^2 + \lambda (||\vec{p}_u||^2 + ||\vec{q}_i||^2)$$

Es posible resolver este problema mediante la técnica de descenso de gradiente, para lo cual debemos encontrar la derivada de la expresión anterior respecto del $\vec{p}_u$ y $\vec{q}_i$. Al hacerlo obtenemos las siguientes ecuaciones de actualización:

$$e_{u,i} = r_{u,i} - \vec{p}_u \cdot \vec{q}_i$$

$$\vec{p}_u = \vec{p}_u + \gamma (e_{u,i} \cdot \vec{q}_i - \lambda \vec{p}_u)$$

$$\vec{q}_i = \vec{q}_i + \gamma (e_{u,i} \cdot \vec{p}_u - \lambda \vec{q}_i)$$

Donde $\lambda$ y $\gamma$ son dos hyper-parámetros del modelo que queremos aprender.

Una vez entrenado el modelo, las matrices $P$ y $Q$ son aprendidas y no necesitan modificarse hasta que la matriz de votaciones cambie sustancialmente. Obtener una predicción una vez el modelo ha aprendido implica, simplemente, realizar el producto escalar de dos vectores de dimensión *k*, que, por lo general, suele ser un valor pequeño.

A este algoritmo se le conoce como ***Probabilistic Matriz Factorization (PMF)***.


## Carga del dataset

Para ilustar mejor el funcionamiento el algoritmo PMF, vamos a desarrollar una implementación del mismo.

Para ello usaremos el dataset de [MovieLens 100K](https://grouplens.org/datasets/movielens/) que contiene 100.000 votos de 943 usuarios sobre 1682 películas. Este dataset ha sido dividido en votaciones de entrenamiento (80%) y votaciones de test (20%). Además, los códigos de usuarios e items, han sido modificados para que comience en 0 y terminen en el número de (usuarios / items) - 1.

Inicialmente definimos algunas constantes que nos serán necesarias durante la codificación del algoritmo:

In [ ]:
import urllib
import random

In [ ]:
NUM_USERS = 943
NUM_ITEMS = 1682

MIN_RATING = 1
MAX_RATING = 5

Y cargamos la matriz con las votaciones de entrenamiento:

In [ ]:
ratings = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 

training_file = urllib.request.urlopen("https://drive.google.com/uc?export=view&id=1S4-sxOEvA3MDivaGf7iFirWqt1H6VtaH")
for line in training_file:
  [u, i, rating] = line.decode("utf-8").split("::")
  ratings[int(u)][int(i)] = int(rating)

Del mismo modo, cargamos la matriz de votaciones de test:

In [ ]:
test_ratings = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 

test_file = urllib.request.urlopen("https://drive.google.com/uc?export=view&id=1LBgTF57DD2NA-petq_FaC1V-h7nrYIh9")
for line in test_file:
  [u, i, rating] = line.decode("utf-8").split("::")
  test_ratings[int(u)][int(i)] = int(rating)

## Inicialización del modelo

Definimos los parámetros necesarios para implementar la factorización matricial mediante PMF.

In [ ]:
NUM_FACTORS = 7
LEARNING_RATE = 0.001 # gamma
REGULARIZATION = 0.1 # lambda

Inicializamos las matrices de factores con valores uniformes aleatorios en el intervalo \[0, 1].

In [ ]:
p = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_USERS)] 
q = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_ITEMS)] 

## Cálculo de las predicciones

Como hemos comentado, calcular la predicción del voto del usuario *u* al item *i* implicar realizar el producto escalar de sus vectores de factores. La siguiente función realiza esta operación:


In [ ]:
def compute_prediction (p_u, q_i):
  prediction = 0
  for k in range(NUM_FACTORS):
    prediction += p_u[k] * q_i[k]
  return prediction

## Aprendizaje de los factores latentes

El proceso de entrenamiento implicar aplicar las operaciones de actualización de las matrices de factores hasta que el algoritmo converja. En general, esta convergencia suele prefijarse como el número de iteraciones que realizamos sobre las operaciones de actualización:

In [ ]:
NUM_ITERATIONS = 10

Es importante resaltar que sólo debemos actualizar las matrices $P$ y $Q$ empleando los votos existentes en la matriz $R$.

El siguiente código ejemplifica el proceso de entrenamiento del algoritmo:

In [9]:
for it in range(NUM_ITERATIONS):
  print("Iteración " + str(it + 1) + " de " + str(NUM_ITERATIONS))
    
  updated_p = list(p) # clone p matrix
  updated_q = list(q) # clone q matrix
    
  for u in range(NUM_USERS):
    for i in range(NUM_ITEMS):
      if ratings[u][i] != None:
                
        prediction = compute_prediction(p[u], q[i])
        rating = ratings[u][i]
        error = rating - prediction
                
        for k in range(NUM_FACTORS):
          updated_p[u][k] += LEARNING_RATE * (error * q[i][k] - REGULARIZATION * p[u][k])
          updated_q[i][k] += LEARNING_RATE * (error * p[u][k] - REGULARIZATION * q[i][k])
        
  p = updated_p
  q = updated_q

Iteración 1 de 10
Iteración 2 de 10
Iteración 3 de 10
Iteración 4 de 10
Iteración 5 de 10
Iteración 6 de 10
Iteración 7 de 10
Iteración 8 de 10
Iteración 9 de 10
Iteración 10 de 10


## Cálculo de las recomendaciones

El cálculo de las recomendaciones, por lo general, simplemente implica seleccionar los *N* items con una predicción más alta. Por ejemplo, si quisiéramos recomendar *N = 3* items a un usuario que tuviera las siguientes predicciones:

|   	| i1 	| i2 	| i3 	| i4 	| i5 	| i6 	| i7 	| i8 	| i9 	| i10 	|
|:-:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|-----	|
| u 	|   	|  2,9 	|    	|  4,7 	|  5,0 	|    	|  1,2 	|    	|   	|  3,1 	|

Se le recomendarían a dicho usuario los items *i5*, *i4* e *i10*.

##Cálculo del MAE

En esta sección vamos a mostrar cómo calcular el error medio absoluto (MAE) de las predicciones realizadas por el algoritmo PMF

Para ello, lo primero que debemos hacer es calcular las predicciones para todos los items que haya recibido una votación de test:

In [ ]:
predictions = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 

# Rellenamos la matriz de predicciones
for u in range(NUM_USERS):
  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None:
      predictions[u][i] = compute_prediction(p[u], q[i])

Y, a continuación, calculamos el MAE:

In [ ]:
def get_user_mae (u):
  mae = 0
  count = 0
  
  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None and predictions[u][i] != None:
      mae += abs(test_ratings[u][i] - predictions[u][i])
      count += 1
  
  if count > 0:
    return mae / count
  else:
    return None

In [ ]:
def get_mae ():
  mae = 0
  count = 0
  
  for u in range(NUM_USERS):
    user_mae = get_user_mae(u)
      
    if user_mae != None:
      mae += user_mae
      count += 1
  
  
  if count > 0:
    return mae / count
  else:
    return None   

In [13]:
mae = get_mae()
print("System MAE = " + str(mae))

System MAE = 0.9049119086407453


## Añadiendo los bias

El modelo descrito anteriormente mejora significativamente la escalabilidad del filtrado colaborativo y, además, incremente notablemente la calidad de las predicciones y recomendaciones. Sin embargo, dicho modelo no se ajusta a la realidad puesto que no refleja los sesgos que los usuarios tienen cuando realizan votaciones.

Parece evidente pensar que no todos los usuarios tienen la misma interpretación de las votaciones. Por ejemplo, existen usuarios más "generosos" con las votaciones que tienden a asignar siempre valoraciones altas y existen usuarios más "tacaños" con las votaciones que tienden a asignar siempre valoraciones más bajas. Que el primer usuario valore un item con 5 y el segundo usuario valore el mismo item con un 4 no quiere decir que al primero le haya gustado más el item. Cada usuario hace su propia interpretación de lo que significan los votos 4 y 5.

Igualmente, existen determinados items que socialmente tienen que gustar y existen otros items que está "mal visto" que gusten. Por ejemplo, resulta extraño que alguien pueda otorgar la nota mínima a *El Padrino* aunque no le haya gustado. La presión social hace que dicha película sea importante, y eso condiciona nuestro voto sobre la misma. Igualmente, resulta extraño que alguien pueda otorgar la nota máxima a *Sharknado* ya que, socialmente, es considerada una película "mala".

Para reflejar este fenómeno dentro de nuestro modelo de factorización matricial, debemos hacer algunas modificaciones sobre el mismo. Para empezar, cambiaremos cómo se calculan las predicciones:

$$\hat{r}_{u,i} = \mu + b_u + b_i + \vec{p}_u \cdot \vec{q}_i$$

Donde $\mu$ representa la votación media de la base de datos, $b_u$ representa el bias (sesgo) del usuario $u$, $b_i$ representa el bias (sesgo) del item $i$ y $\vec{p}_u \cdot \vec{q}_i$ simboliza la interacción entre el usuario $u$ y el item $i$. 

De este modo, la predicción será calculada como la media de la base de datos, +/- un ajuste en función de cómo suele vota el usuario, +/- un ajuste de cómo suele votarse el item, y +/- la interacción entre el usuario y el item.

Debido a este cambio, la función a minimizar es ahora la siguiente:

$$\min_{b_u, b_i,p,q} \sum_{(u,i) \in R} ( r_{u,i} - \mu - b_u - b_i - \vec{p}_u \cdot \vec{q}_i)^2 + \lambda (||\vec{p}_u||^2 + ||\vec{q}_i||^2 + b_u^2 + b_i^2)$$

A la que, tras aplicar la derivada respecto de $b_u$, $q_i$, $\vec{p}_u$ y $\vec{q}_i$ obtenemos:

$$e_{u,i} = r_{u,i} - \mu - b_u - b_i - \vec{p}_u \cdot \vec{q}_i$$

$$b_u = b_u + \gamma (e_{u,i} - \lambda b_u)$$

$$b_i = b_i + \gamma (e_{u,i} - \lambda b_i)$$

$$\vec{p}_u = \vec{p}_u + \gamma (e_{u,i} \cdot \vec{q}_i - \lambda \vec{p}_u)$$

$$\vec{q}_i = \vec{q}_i + \gamma (e_{u,i} \cdot \vec{p}_u - \lambda \vec{q}_i)$$

Definimos una nueva función para calcular las predicciones:

In [ ]:
def compute_biased_prediction (avg, b_u, b_i, p_u, q_i):
  deviation = 0
  for k in range(NUM_FACTORS):
    deviation += p_u[k] * q_i[k]
        
  prediction = avg + b_u + b_i + deviation
  return prediction

Calculamos el voto medio:

In [ ]:
rating_average = 0
rating_count = 0

for u in range(NUM_USERS):
  for i in range(NUM_ITEMS):
    if ratings[u][i] != None:
      rating_average += ratings[u][i]
      rating_count += 1
      
rating_average /= rating_count    

Reiniciamos las matrices de factores y los vectores de bias con valores aleatorios en el intervalo \[0, 1]:

In [ ]:
p = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_USERS)] 
q = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_ITEMS)] 

bu = [random.random() for _ in range(NUM_USERS)]
bi = [random.random() for _ in range(NUM_ITEMS)]

Y volvemos a entrenar nuestro modelo:

In [17]:
for it in range(NUM_ITERATIONS):
  print("Iteración " + str(it + 1) + " de " + str(NUM_ITERATIONS))
    
  updated_p = list(p) # clone p matrix
  updated_q = list(q) # clone q matrix
    
  updated_bu = list(bu) # clone bu vector
  updated_bi = list(bi) # clone bi vector
    
  for u in range(NUM_USERS):
    for i in range(NUM_ITEMS):
      if ratings[u][i] != None:
                
        prediction = compute_biased_prediction(rating_average, bu[u], bi[i], p[u], q[i])                
        rating = ratings[u][i]
        error = rating - prediction
                
        for k in range(NUM_FACTORS):
          updated_p[u][k] += LEARNING_RATE * (error * q[i][k] - REGULARIZATION * p[u][k])
          updated_q[i][k] += LEARNING_RATE * (error * p[u][k] - REGULARIZATION * q[i][k])
                    
        updated_bu[u] += LEARNING_RATE * (error - REGULARIZATION * bu[u])
        updated_bi[i] += LEARNING_RATE * (error - REGULARIZATION * bi[i])

        
  p = updated_p
  q = updated_q
    
  bu = updated_bu
  bi = updated_bi

Iteración 1 de 10
Iteración 2 de 10
Iteración 3 de 10
Iteración 4 de 10
Iteración 5 de 10
Iteración 6 de 10
Iteración 7 de 10
Iteración 8 de 10
Iteración 9 de 10
Iteración 10 de 10


Calculamos las nuevas predicciones:

In [ ]:
predictions = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 

for u in range(NUM_USERS):
  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None:
      predictions[u][i] = compute_biased_prediction(rating_average, bu[u], bi[i], p[u], q[i])

Y calculamos el nuevo MAE:

In [19]:
mae = get_mae()
print("System MAE = " + str(mae))

System MAE = 0.9597400939551685


## Referencias

Mnih, A., & Salakhutdinov, R. R. (2008). **Probabilistic matrix factorization**. In Advances in neural information processing systems (pp. 1257-1264).

Koren, Y., Bell, R., & Volinsky, C. (2009). **Matrix factorization techniques for recommender systems**. Computer, (8), 30-37.

---

*Este documento ha sido desarrollado por **Fernando Ortega**. Dpto. Sistemas Informáticos, ETSI de Sistemas Informáticos, Universidad Politécnica de Madrid.*

*Última actualización: Marzo de 2024*


<p xmlns:cc="http://creativecommons.org/ns#" >This work is licensed under <a href="http://creativecommons.org/licenses/by-nc/4.0/?ref=chooser-v1" target="_blank" rel="license noopener noreferrer" style="display:inline-block;">CC BY-NC 4.0<img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/cc.svg?ref=chooser-v1"><img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/by.svg?ref=chooser-v1"><img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/nc.svg?ref=chooser-v1"></a></p>